In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import SimpleITK as sitk
from tqdm import tqdm

In [28]:
def get_diameter(row):
    # return (row["w"]+row["h"]+row["d"])/3
    return min(row["w"], row["h"], row["d"])

# Stats on false negatives 

In [29]:
path_vessel_seg = Path(
    "/home/ceballosarroyo.a/workspace/datasets/aneurysm/internal_test_0.4_crop_vessel_v2"
)
path_vessel_seg_edt = Path(
    "/home/ceballosarroyo.a/workspace/datasets/aneurysm/internal_test_0.4_crop_vessel_edt_v2"
)


files_vessel_seg = list(path_vessel_seg.glob("*.nii.gz"))
files_vessel_seg_edt = list(path_vessel_seg_edt.glob("*.nii.gz"))

In [30]:
path_preds = "/home/ceballosarroyo.a/workspace/medical/cta-det2/outputs/adeform_decoder_only_non_rec_crop_vessel_pe_gpe_v2/inference_56k/predict.csv"
df_preds = pd.read_csv(path_preds)

In [31]:
df_preds.head()

,seriesuid,probability,coordZ,coordY,coordX,d,h,w,intersection_art,intersection_vein,distance
0,Ts0001.nii.gz,0.863451,194.49065,389.67150,413.88654,13.835364,15.982091,15.982846,0.56165,0.0,0.500912
1,Ts0001.nii.gz,0.746006,133.32965,416.58493,578.43555,11.540600,13.419129,13.588934,NaN,NaN,NaN
2,Ts0001.nii.gz,0.714957,160.85077,430.25240,418.74075,12.876307,14.913800,14.943136,NaN,NaN,NaN
3,Ts0001.nii.gz,0.655347,183.01541,380.73376,423.84305,13.536043,16.478287,16.227999,NaN,NaN,NaN
4,Ts0001.nii.gz,0.545710,194.24483,430.60240,385.80518,11.976373,15.650705,15.548584,NaN,NaN,NaN


In [32]:
intersections = []
intersections_vein = []
distances = []
for case_name in tqdm(df_preds["seriesuid"].unique()):

    df_preds_case = df_preds[df_preds["seriesuid"] == case_name]

    header_seg = sitk.ReadImage(str(path_vessel_seg / f"{case_name}"))
    header_edt = sitk.ReadImage(str(path_vessel_seg_edt / f"{case_name}"))
    edt_array = sitk.GetArrayFromImage(header_edt)
    seg_array = sitk.GetArrayFromImage(header_seg)
    seg_array_vein = (seg_array == 2).astype(np.uint8)
    seg_array_ves = (seg_array == 1).astype(np.uint8)
    for _, row in df_preds_case.iterrows():
        if row["probability"] < 0.8:
            intersections.append(None)
            distances.append(None)
            intersections_vein.append(None)
            continue
        pred_array = np.zeros_like(seg_array)
        coordZ, coordY, coordX = (
            int(row["coordZ"]),
            int(row["coordY"]),
            int(row["coordX"]),
        )
        h, w, d = int(row["h"]), int(row["w"]), int(row["d"])
        pred_array[
            max(0, int(coordZ - d // 2)) : min(
                pred_array.shape[0], int(coordZ + d // 2)
            ),
            max(0, int(coordY - h // 2)) : min(
                pred_array.shape[1], int(coordY + h // 2)
            ),
            max(0, int(coordX - w // 2)) : min(
                pred_array.shape[2], int(coordX + w // 2)
            ),
        ] = 1
        # get intersection over minimum
        intersection = np.sum(pred_array * seg_array_ves)

        minimum = np.sum(pred_array)
        intersection_vein = np.sum(pred_array * seg_array_vein)
        intersections_vein.append(intersection_vein / minimum)
        avg_distance = edt_array[coordZ, coordY, coordX]
        intersections.append(intersection / minimum)
        distances.append(avg_distance)

100%|██████████| 152/152 [31:27<00:00, 12.42s/it]


In [33]:
backup_vein = intersections_vein.copy()

In [18]:
new_intersection_vein = []

for i in range(len(intersections)):
    if intersections[i] is None:
        new_intersection_vein.append(None)
    else:
        # pop the first element from vein
        new_intersection_vein.append(intersections_vein.pop(0))

In [34]:
df_preds["intersection_art"] = intersections
df_preds["intersection_vein"] = new_intersection_vein
df_preds["distance"] = distances

df_preds.to_csv(
    path_preds,
    index=False,
)

In [11]:
seg_array.max()

1.0

In [47]:
repo_root = Path("/home/ceballosarroyo.a/workspace/medical/cta-det2")

iou = '0.2'
exp = 'deform_decoder_only_non_rec_BEST_cropinf'
chkpt = "final"

#exp = 'dense_bn_64_infer'
#chkpt = 'hieu'

path_test_annot = repo_root / "labels/internal_test_crop_0.4.csv"
path_results = repo_root / f"outputs/{exp}/iou{iou}_froc_{chkpt}/model_detections.csv"



In [48]:
df_annot = pd.read_csv(path_test_annot)
df_results = pd.read_csv(path_results)

df_annot.columns

Index(['seriesuid', 'coordX', 'coordY', 'coordZ', 'w', 'h', 'd', 'lesion'], dtype='object')

In [49]:
df_annot["detected"] = df_results["detected"]
df_annot["diameter"] = df_annot.apply(get_diameter, axis=1)
df_annot_true = df_annot[df_annot["detected"] == True]
df_annot_false = df_annot[df_annot["detected"] == False]
df_annot_true[["w","h", "d"]].mean(), df_annot_false[["w","h", "d"]].mean()

(w    6.681318
 h    6.463972
 d    5.676055
 dtype: float64,
 w     9.813750
 h    10.290000
 d    14.399989
 dtype: float64)

In [52]:
def diameter_wise_clf(diameter):
    if diameter < 3:
        return "small"
    elif diameter >= 3 and diameter < 7:
        return "medium"
    else: 
        return "large"


In [53]:
df_annot["size"] = df_annot["diameter"].apply(diameter_wise_clf)

# count detections per size 

df_annot[df_annot["detected"]==True]["size"].value_counts()

size
medium    81
large     24
small     16
Name: count, dtype: int64

In [54]:
df_annot[df_annot["detected"]==False]["size"].value_counts()

size
medium    3
large     1
small     1
Name: count, dtype: int64